In [1]:
"""
SHAP para explicación de modelos de visión - Versión secuencial explicativa

Este script demuestra cómo utilizar SHAP (SHapley Additive exPlanations) para
explicar las predicciones de un modelo de visión ResNet entrenado para clasificación
de imágenes.

Cada sección está comentada para explicar su propósito y funcionamiento.
"""

import torch
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import os
import shap
from PIL import Image
import argparse
import random
import matplotlib.gridspec as gridspec

# ---------------------------------------------------------------------
# SECCIÓN 1: CONFIGURACIÓN INICIAL
# ---------------------------------------------------------------------
print("="*70)
print("SHAP PARA EXPLICACIÓN DE MODELOS DE VISIÓN - TUTORIAL SECUENCIAL")
print("="*70)

# Configurar Parámetros
# Establece IMAGE_PATH específico para analizar una imagen concreta, o None para selección automática
IMAGE_PATH = None  # Ejemplo: "../work/val_images_classes/0/imagen.jpg"

# Establece CLASS_ID para seleccionar una imagen aleatoria de una clase específica (0-4), o None
CLASS_ID = None  # Opciones: 0, 1, 2, 3, 4 o None

# Establece en True para seleccionar una imagen completamente aleatoria
RANDOM_IMAGE = True  

# Número de evaluaciones máximas para SHAP (aumentar para resultados más detallados, pero más lento)
MAX_EVALS = 500  

# Rutas y configuración básica
MODEL_PATH = "../work/optuna_temp_artifacts/04 ResNet Augment_1.0.0_20250416_002028.pth"
IMAGE_SIZE = 299
OUTPUT_DIR = "shap_results"
VAL_IMAGES_DIR = "../work/val_images_classes"

# Crear directorio para resultados
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Definir nombres de clases
class_names = ['0', '1', '2', '3', '4']

print("\n[PASO 1] Configuración completada.")
print(f"- Modelo a explicar: {MODEL_PATH}")
print(f"- Resultados se guardarán en: {OUTPUT_DIR}")
if MAX_EVALS != 500:
    print(f"- Usando {MAX_EVALS} evaluaciones para SHAP (mayor detalle)")

# ---------------------------------------------------------------------
# SECCIÓN 2: CARGAR EL MODELO
# ---------------------------------------------------------------------
print("\n" + "="*50)
print("[PASO 2] CARGANDO EL MODELO")
print("="*50)

# Determinar si hay GPU disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"- Utilizando dispositivo: {device}")

# Cargar el modelo previamente entrenado
try:
    model = torch.load(MODEL_PATH, map_location=device)
    model.eval()  # Poner el modelo en modo evaluación
    print("- Modelo cargado correctamente.")
except Exception as e:
    print(f"ERROR al cargar el modelo: {e}")
    import sys
    sys.exit(1)

# ---------------------------------------------------------------------
# SECCIÓN 3: DEFINIR FUNCIONES DE PREPROCESAMIENTO
# ---------------------------------------------------------------------
print("\n" + "="*50)
print("[PASO 3] PREPARANDO FUNCIONES DE PREPROCESAMIENTO")
print("="*50)

# Transformaciones para normalizar las imágenes
# (estas deben coincidir con las usadas durante el entrenamiento)
val_transforms = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

print("- Definidas transformaciones para preprocesamiento de imágenes:")
print("  * Redimensionar a", IMAGE_SIZE, "x", IMAGE_SIZE)
print("  * Convertir a tensor")
print("  * Normalizar con valores ImageNet")

# ---------------------------------------------------------------------
# SECCIÓN 4: FUNCIONES PARA PREDICCIÓN Y SHAP
# ---------------------------------------------------------------------
print("\n" + "="*50)
print("[PASO 4] DEFINIENDO FUNCIONES PARA PREDICCIÓN Y SHAP")
print("="*50)

def load_image(image_path):
    """Cargar y preprocesar una imagen para predicción con el modelo"""
    try:
        image = Image.open(image_path).convert('RGB')
        return val_transforms(image).unsqueeze(0).to(device), np.array(image)
    except Exception as e:
        print(f"ERROR al cargar la imagen {image_path}: {e}")
        return None, None

def predict_class(image_tensor):
    """Predecir la clase de una imagen con el modelo cargado"""
    with torch.no_grad():  # No necesitamos gradientes para predicción
        output = model(image_tensor)
        probabilities = torch.nn.functional.softmax(output, dim=1)
        predicted_class = torch.argmax(probabilities, dim=1).item()
        probs = probabilities[0].cpu().numpy()
    
    return predicted_class, probs

def f(x):
    """Función para SHAP que toma imágenes y devuelve predicciones
    
    Esta función es necesaria para el explainer de SHAP y debe:
    1. Recibir imágenes en formato numpy
    2. Preprocesarlas adecuadamente
    3. Pasarlas por el modelo
    4. Devolver las probabilidades para cada clase
    """
    # Normalizar si es necesario
    if x.max() > 1:
        x = x / 255.0
    
    # Convertir a tensor de PyTorch
    # Formato de entrada: NHWC (lote, alto, ancho, canales)
    x_tensor = torch.tensor(x, dtype=torch.float32)
    # Cambiar formato a NCHW (lote, canales, alto, ancho) que es lo que espera PyTorch
    x_tensor = x_tensor.permute(0, 3, 1, 2)
    
    # Normalizar con los mismos valores que en el entrenamiento
    mean = torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1)
    x_tensor = (x_tensor - mean) / std
    
    # Predicción
    with torch.no_grad():
        x_tensor = x_tensor.to(device)
        outputs = model(x_tensor)
        probs = torch.nn.functional.softmax(outputs, dim=1)
    
    return probs.cpu().numpy()

def select_random_image(class_id=None):
    """Seleccionar una imagen aleatoria, ya sea de una clase específica o de cualquier clase"""
    if class_id is not None:
        # Buscar en la clase específica
        class_dir = os.path.join(VAL_IMAGES_DIR, str(class_id))
        if not os.path.exists(class_dir):
            print(f"ERROR: No se encontró el directorio para la clase {class_id}")
            return None, None
            
        image_files = os.listdir(class_dir)
        if not image_files:
            print(f"ERROR: No hay imágenes en la clase {class_id}")
            return None, None
            
        # Seleccionar una imagen aleatoria
        selected_image = random.choice(image_files)
        return os.path.join(class_dir, selected_image), class_id
    else:
        # Seleccionar cualquier clase
        class_dirs = [os.path.join(VAL_IMAGES_DIR, c) for c in class_names if os.path.exists(os.path.join(VAL_IMAGES_DIR, c))]
        if not class_dirs:
            print("ERROR: No se encontraron directorios de clases")
            return None, None
            
        # Seleccionar una clase aleatoria
        selected_class_dir = random.choice(class_dirs)
        class_id = int(os.path.basename(selected_class_dir))
        
        image_files = os.listdir(selected_class_dir)
        if not image_files:
            print(f"ERROR: No hay imágenes en {selected_class_dir}")
            return None, None
            
        # Seleccionar una imagen aleatoria
        selected_image = random.choice(image_files)
        return os.path.join(selected_class_dir, selected_image), class_id

def custom_image_plot(shap_values, pixel_values, save_path, show=False):
    """Versión personalizada de shap.image_plot con la barra de colores en la parte inferior
    
    Esta función crea una visualización SHAP más limpia y estética que la función
    estándar, con la barra de colores posicionada en la parte inferior y mayor
    visibilidad de la imagen original.
    
    Argumentos:
        shap_values: Valores SHAP calculados por el explainer
        pixel_values: Valores de los píxeles de la imagen original
        save_path: Ruta donde guardar la visualización
        show: Si True, muestra la visualización además de guardarla
    """
    # Obtenemos las clases
    if isinstance(shap_values.output_names[0], str):
        class_names = [name for name in shap_values.output_names]
    else:
        class_names = [str(name) for name in shap_values.output_names]
    
    # Configuramos tamaño de figura y aspecto
    x = pixel_values
    fig_size = np.array([3 * (len(class_names) + 1), 2.5 * (x.shape[0] + 1)])
    if fig_size[0] > 20:
        fig_size *= 20 / fig_size[0]
    
    # Creamos una figura con un grid personalizado
    fig = plt.figure(figsize=fig_size)
    grid = gridspec.GridSpec(x.shape[0] + 1, len(class_names) + 1, height_ratios=[20] * x.shape[0] + [1])
    
    print("- Generando visualización para", len(class_names), "clases")
    
    # Para cada imagen
    for row in range(x.shape[0]):
        x_curr = x[row].copy()
        
        # Normalizamos si es necesario
        if x_curr.max() > 1 or x_curr.min() < 0:
            x_curr = (x_curr - x_curr.min()) / (x_curr.max() - x_curr.min())
        
        # Imagen original
        ax_img = plt.subplot(grid[row, 0])
        ax_img.imshow(x_curr)
        ax_img.set_title("Imagen Original")
        ax_img.axis('off')
        
        # Valores absolutos para establecer escala de colores
        abs_vals = []
        for i in range(len(class_names)):
            sv = shap_values.values[row, :, :, :, i]
            abs_vals.append(np.abs(sv))
        abs_vals = np.stack(abs_vals, 0).flatten()
        max_val = np.nanpercentile(abs_vals, 99.9)
        
        # Para cada clase, mostramos los valores SHAP
        for i in range(len(class_names)):
            sv = shap_values.values[row, :, :, :, i]
            
            ax_shap = plt.subplot(grid[row, i + 1])
            
            # Imagen original en gris con mayor visibilidad
            gray_img = np.mean(x_curr, axis=2)
            ax_shap.imshow(gray_img, cmap=plt.cm.gray, alpha=0.8)  # Imagen base con buena visibilidad
            
            # Valores SHAP superpuestos con transparencia
            im = ax_shap.imshow(sv.sum(axis=2), cmap=plt.cm.RdBu_r, vmin=-max_val, vmax=max_val, alpha=0.4)
            class_title = f"Clase {class_names[i]}"
            if int(class_names[i]) == predicted_class:
                class_title += " (Predicha)"
            ax_shap.set_title(class_title)
            ax_shap.axis('off')
    
    # Añadir barra de colores en la parte inferior
    cbar_ax = plt.subplot(grid[x.shape[0], :])
    cb = plt.colorbar(im, cax=cbar_ax, orientation='horizontal')
    cb.set_label('SHAP value (rojo = aumenta probabilidad, azul = disminuye probabilidad)')
    
    plt.tight_layout()
    plt.savefig(save_path, bbox_inches='tight', dpi=150)
    
    if show:
        plt.show()
    else:
        plt.close()
    
    print(f"- Visualización guardada en: {save_path}")

print("- Funciones definidas:")
print("  * load_image: Para cargar y preprocesar imágenes")
print("  * predict_class: Para obtener predicciones del modelo")
print("  * f: Función especial para el explainer de SHAP") 
print("  * select_random_image: Para seleccionar imágenes aleatorias para análisis")
print("  * custom_image_plot: Para crear visualizaciones SHAP más estéticas")

# ---------------------------------------------------------------------
# SECCIÓN 5: SELECCIONAR UNA IMAGEN PARA ANALIZAR
# ---------------------------------------------------------------------
print("\n" + "="*50)
print("[PASO 5] SELECCIONANDO UNA IMAGEN PARA ANALIZAR")
print("="*50)

true_class = None

# Si ya se proporcionó una imagen específica, usarla
if IMAGE_PATH:
    if not os.path.exists(IMAGE_PATH):
        print(f"ERROR: La imagen {IMAGE_PATH} no existe")
        import sys
        sys.exit(1)
    print(f"- Usando imagen proporcionada: {IMAGE_PATH}")
    # No podemos conocer la clase real en este caso
    true_class = None
    
# Si se especificó una clase, seleccionar una imagen aleatoria de esa clase
elif CLASS_ID is not None:
    IMAGE_PATH, true_class = select_random_image(CLASS_ID)
    if IMAGE_PATH is None:
        print(f"ERROR: No se pudo seleccionar una imagen de la clase {CLASS_ID}")
        import sys
        sys.exit(1)
    print(f"- Seleccionada imagen aleatoria de clase {CLASS_ID}: {IMAGE_PATH}")
    
# En otro caso, seleccionar una imagen completamente aleatoria
else:
    IMAGE_PATH, true_class = select_random_image()
    if IMAGE_PATH is None:
        print("ERROR: No se pudo seleccionar una imagen aleatoria")
        import sys
        sys.exit(1)
    print(f"- Seleccionada imagen aleatoria: {IMAGE_PATH}")

if true_class is not None:
    print(f"- Clase real de la imagen: {true_class}")

# ---------------------------------------------------------------------
# SECCIÓN 6: PREDECIR CLASE DE LA IMAGEN
# ---------------------------------------------------------------------
print("\n" + "="*50)
print("[PASO 6] PREDICIENDO CLASE DE LA IMAGEN")
print("="*50)

# Cargar la imagen y hacer la predicción
img_tensor, img_original = load_image(IMAGE_PATH)
if img_tensor is None:
    print("ERROR: No se pudo cargar la imagen.")
    import sys
    sys.exit(1)

predicted_class, probabilities = predict_class(img_tensor)

print(f"- Imagen: {os.path.basename(IMAGE_PATH)}")
print(f"- Clase predicha: {predicted_class}")
if true_class is not None:
    if predicted_class == true_class:
        print(f"- ¡Predicción correcta! La clase real es: {true_class}")
    else:
        print(f"- Predicción incorrecta. La clase real es: {true_class}")

print("- Probabilidades por clase:")
for i, prob in enumerate(probabilities):
    print(f"  * Clase {i}: {prob:.4f}")

# Guardar imagen original
plt.figure(figsize=(8, 8))
plt.imshow(img_original)
title = f"Imagen Original (Clase predicha: {predicted_class}"
if true_class is not None:
    title += f", Clase real: {true_class})"
else:
    title += ")"
plt.title(title)
plt.axis('off')
original_path = os.path.join(OUTPUT_DIR, f"original_{os.path.basename(IMAGE_PATH)}")
plt.savefig(original_path, bbox_inches='tight')
plt.close()
print(f"- Imagen original guardada en: {original_path}")

# ---------------------------------------------------------------------
# SECCIÓN 7: CONFIGURAR Y EJECUTAR SHAP
# ---------------------------------------------------------------------
print("\n" + "="*50)
print("[PASO 7] CONFIGURANDO SHAP PARA EXPLICAR LA PREDICCIÓN")
print("="*50)

# Convertir imagen a formato numpy y preparar para SHAP
img_np = np.array(img_original)
img_batch = np.expand_dims(img_np, 0)

# Crear un masker (máscara) para SHAP
# El masker define cómo se ocultarán partes de la imagen para calcular contribuciones
print("- Creando masker para SHAP con método 'blur'...")
masker = shap.maskers.Image("blur(16,16)", img_np.shape)
print("  * Este masker difuminará partes de la imagen para ver cómo afecta a la predicción")

# Crear el explainer de SHAP
print("- Inicializando SHAP Partition explainer...")
explainer = shap.Explainer(f, masker, output_names=class_names)
print("  * El Partition explainer divide la imagen en segmentos para analizar contribuciones")

# Calcular valores SHAP
# Esto puede tomar tiempo ya que SHAP necesita evaluar el modelo muchas veces
print("\n- Calculando valores SHAP (esto puede tomar tiempo)...")
print(f"  * Evaluaciones máximas: {MAX_EVALS}")
print("  * Tamaño de lote: 50")

# Ordenamos las salidas según la probabilidad para explicar las clases más relevantes
indices_ordenados = np.argsort(-probabilities)[:5]  # Top 5 clases
print("  * Explicando las 5 clases con mayor probabilidad:", indices_ordenados)

# Ejecutar SHAP
shap_values = explainer(img_batch, max_evals=MAX_EVALS, batch_size=50, outputs=indices_ordenados.tolist())
print("- Cálculo de valores SHAP completado!")

# ---------------------------------------------------------------------
# SECCIÓN 8: VISUALIZAR RESULTADOS SHAP
# ---------------------------------------------------------------------
print("\n" + "="*50)
print("[PASO 8] VISUALIZANDO RESULTADOS DE SHAP")
print("="*50)

# Generar visualización personalizada SHAP
print("- Generando visualización SHAP mejorada...")
shap_output_path = os.path.join(OUTPUT_DIR, f"shap_standard_{os.path.basename(IMAGE_PATH)}")
custom_image_plot(shap_values, -img_batch, shap_output_path, show=False)

# Crear visualización superpuesta para la clase predicha
print("\n- Generando mapa de calor superpuesto para la clase predicha...")

plt.figure(figsize=(16, 8))

# Imagen original
plt.subplot(1, 2, 1)
plt.imshow(img_np)
title = "Imagen Original"
if true_class is not None:
    title += f" (Clase real: {true_class})"
plt.title(title, fontsize=16)
plt.axis('off')

# Encontrar el índice correcto para la clase predicha
idx = None
for i, name in enumerate(shap_values.output_names):
    if int(name) == predicted_class:
        idx = i
        break

if idx is not None:
    # Superponer valores SHAP
    plt.subplot(1, 2, 2)
    
    # Obtener valores para la clase predicha
    class_values = shap_values.values[0, :, :, :, idx]
    
    # Agregar a través de canales para visualización 2D
    importance = np.abs(class_values).sum(axis=2)
    
    # Normalizar para visualización
    importance = importance / importance.max()
    
    # Superponer con transparencia
    plt.imshow(img_np)
    plt.imshow(importance, cmap='hot', alpha=0.3)
    plt.title(f"SHAP Clase {predicted_class} Superpuesto", fontsize=16)
    plt.axis('off')
    
    # Guardar
    overlay_path = os.path.join(OUTPUT_DIR, f"shap_overlay_{os.path.basename(IMAGE_PATH)}")
    plt.tight_layout()
    plt.savefig(overlay_path, bbox_inches='tight', dpi=150)
    plt.close()
    print(f"- Mapa de calor superpuesto guardado en: {overlay_path}")
else:
    print(f"ERROR: No se encontraron valores SHAP para la clase {predicted_class}")

# ---------------------------------------------------------------------
# SECCIÓN 9: EXPLICACIÓN DE LOS RESULTADOS
# ---------------------------------------------------------------------
print("\n" + "="*50)
print("[PASO 9] EXPLICACIÓN DE LOS RESULTADOS")
print("="*50)

print("""
¿Qué significan los resultados SHAP?

1. Los colores ROJOS indican características que AUMENTAN la probabilidad de la clase.
2. Los colores AZULES indican características que DISMINUYEN la probabilidad de la clase.
3. La intensidad del color indica la MAGNITUD de la contribución.

En la visualización estándar SHAP:
- Cada fila representa una imagen de entrada.
- Cada columna representa una clase diferente.
- La barra de colores muestra la escala de contribución.

En el mapa de calor superpuesto:
- Se muestra la contribución específica para la clase predicha.
- Las áreas más brillantes tienen mayor influencia en la predicción.
""")

# ---------------------------------------------------------------------
# SECCIÓN 10: CONCLUSIÓN
# ---------------------------------------------------------------------
print("\n" + "="*50)
print("[PASO 10] CONCLUSIÓN")
print("="*50)

print(f"""
El análisis SHAP nos ha permitido explicar por qué el modelo clasificó la imagen como
clase {predicted_class} con una probabilidad de {probabilities[predicted_class]:.4f}.

Todas las visualizaciones se han guardado en:
{OUTPUT_DIR}

SHAP es una herramienta poderosa para:
1. Entender qué características del input influyen en la predicción
2. Verificar si el modelo se está enfocando en las partes correctas de la imagen
3. Detectar posibles sesgos o problemas en el modelo
4. Aumentar la confianza en las predicciones del modelo

Para más información sobre SHAP, visite: https://github.com/slundberg/shap
""")

print("\n" + "="*70)
print("FIN DEL TUTORIAL")
print("="*70) 

/home/tomas/.cache/pypoetry/virtualenvs/exp-tomi-mFFVELsD-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SHAP PARA EXPLICACIÓN DE MODELOS DE VISIÓN - TUTORIAL SECUENCIAL

[PASO 1] Configuración completada.
- Modelo a explicar: ../work/optuna_temp_artifacts/04 ResNet Augment_1.0.0_20250416_002028.pth
- Resultados se guardarán en: shap_results

[PASO 2] CARGANDO EL MODELO
- Utilizando dispositivo: cuda
- Modelo cargado correctamente.

[PASO 3] PREPARANDO FUNCIONES DE PREPROCESAMIENTO
- Definidas transformaciones para preprocesamiento de imágenes:
  * Redimensionar a 299 x 299
  * Convertir a tensor
  * Normalizar con valores ImageNet

[PASO 4] DEFINIENDO FUNCIONES PARA PREDICCIÓN Y SHAP
- Funciones definidas:
  * load_image: Para cargar y preprocesar imágenes
  * predict_class: Para obtener predicciones del modelo
  * f: Función especial para el explainer de SHAP
  * select_random_image: Para seleccionar imágenes aleatorias para análisis
  * custom_image_plot: Para crear visualizaciones SHAP más estéticas

[PASO 5] SELECCIONANDO UNA IMAGEN PARA ANALIZAR
- Seleccionada imagen aleatoria: ../w